In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import csv
import klib 
import matplotlib.pyplot as plt
import datetime

## Initial preprocessing

In [ ]:
health_df_2019 = pd.read_csv('2019_healthcare.csv')
health_df_2021 = pd.read_csv('2021_healthcare.csv')

In [ ]:
health_df_2019.columns

In [ ]:
health_df_2019

In [ ]:
health_df_2021.columns

In [ ]:
health_df_2021

In [ ]:
for col in health_df_2021.columns:
    if col not in health_df_2019.columns:
        print(col)

In [ ]:
for col in health_df_2019.columns:
    if col not in health_df_2021.columns:
        print(col)

In [ ]:
health_df_2019.drop(['FUP30 Total', 'FUP30 Wound'], axis = 1, inplace = True)

In [ ]:
health_df_2021.drop(['FUP30 EQ5D-5L', 'FUP120 Date','FUP120 Rep By', 'FUP120 Res Status', 'FUP120 RIP Date'], axis = 1, inplace=True)

In [ ]:
health_df_2019['year'] = '2019'
health_df_2021['year'] = '2021'

In [ ]:
frames = [health_df_2019, health_df_2021]
full_data = pd.concat(frames)

In [ ]:
full_data = klib.clean_column_names(full_data)

In [ ]:
full_data

In [ ]:
full_data.isnull().sum()

## Parse fup_30_re_adm's

In [ ]:
full_data['fup30_re_adm'].unique()

In [ ]:
full_data['fup30_re_adm_29'] = full_data['fup30_re_adm_29'].fillna('Null')
full_data['fup30_re_adm'] = full_data['fup30_re_adm'].fillna(full_data['fup30_re_adm_29'])

In [ ]:
full_data.drop(['fup30_re_adm_29'], inplace = True, axis = 1)

## Impute missing values on other feature columns

* pre_fx_indoor         
* pre_fx_outdoor        
* pre_fx_shopping 
* ger_date

In [ ]:
full_data['pre_fx_shopping'] = full_data['pre_fx_shopping'].fillna('Null')
full_data['pre_fx_outdoor'] = full_data['pre_fx_outdoor'].fillna('Null')
full_data['pre_fx_indoor'] = full_data['pre_fx_indoor'].fillna('Null')

In [ ]:
percentage_of_non_missing_dates = (full_data['ger_date'].notnull().sum())/full_data['ger_date'].isnull().sum() * 100
print('Number of non missing dates = {} %'.format(round(percentage_of_non_missing_dates,2)))

#### Since number of non missing dates are too high, this column has no significant impact on the target variable, hence dropped

In [ ]:
full_data.drop(['ger_date'], axis = 1, inplace= True)

* asa_grade            
* surg_delay           
* surg_delay_other    

In [ ]:
full_data['asa_grade'].dtypes

In [ ]:
full_data['asa_grade'] = pd.Categorical(full_data.asa_grade)
full_data['asa_grade'] = full_data['asa_grade'].fillna(full_data['asa_grade'].mode().iloc[0])

In [ ]:
full_data['surg_delay'] = full_data['surg_delay'].fillna(full_data['surg_delay'].mode().iloc[0])

In [ ]:
full_data['surg_delay_other'] = full_data['surg_delay_other'].fillna('Null')

* re_op_30             
* operation_2   
* bone_med_change     

In [ ]:
full_data['re_op_30'] = full_data['re_op_30'].fillna('None')

In [ ]:
percentage_of_non_missing_op2 = (full_data['operation_2'].notnull().sum())/full_data['operation_2'].isnull().sum() * 100
print('Number of non missing operation_2 = {} %'.format(round(percentage_of_non_missing_op2,2)))

### Since number of non missing op2 are too high, this column has no significant impact on the target variable, hence dropped

In [ ]:
full_data.drop(['operation_2'], axis = 1, inplace= True)

In [ ]:
full_data['bone_med_change'] = full_data['bone_med_change'].fillna('None')

* fup30_date     
* fup30_res_status    
* fup30_rip_date      

### fup date not significance, as even when there is no date, the followup still happens

In [ ]:
full_data.drop(['fup30_date'], axis = 1, inplace= True)

In [ ]:
full_data['fup30_res_status'] = full_data['fup30_res_status'].fillna('Unknown')

In [ ]:
full_data['fup30_rip_date'] = full_data['fup30_rip_date'].fillna('Null')

## Replace rip date with is_rip (0 means not dead, 1 means dead)

In [ ]:
fup_isRip = []
for data in full_data['fup30_rip_date']:
    if data == 'Null':
        fup_isRip.append('0')
    else:
        fup_isRip.append('1')   
    

In [ ]:
full_data['fup_isRip'] = fup_isRip

In [ ]:
full_data.drop(['fup30_rip_date'], axis = 1, inplace= True)

* fup30_indoor        
* fup30_outdoor       
* fup30_shopping      

In [ ]:
full_data['fup30_indoor'] = full_data['fup30_indoor'].fillna('Null')
full_data['fup30_outdoor'] = full_data['fup30_outdoor'].fillna('Null')
full_data['fup30_shopping'] = full_data['fup30_shopping'].fillna('Null')

* fup30_re_op        
* fup30_bone_med 

In [ ]:
full_data['fup30_re_op'] = full_data['fup30_re_op'].fillna('None')

## Replace fup30_re_op with fup30_re_op_flag (0 means no reoperate, 1 means reoperate)

In [ ]:
fup30_re_op_flag = []
for data in full_data['fup30_re_op']:
    if data == 'None':
        fup30_re_op_flag.append('0')
    else:
        fup30_re_op_flag.append('1')   

In [ ]:
full_data['fup30_re_op_flag'] = fup30_re_op_flag

In [ ]:
full_data.drop(['fup30_re_op'], axis = 1, inplace= True)

In [ ]:
full_data['fup30_bone_med'].unique()

In [ ]:
full_data['fup30_bone_med'] =  full_data['fup30_bone_med'].fillna('Unknown')

## fup30_pain_as and fup30_pain parsing

In [ ]:
full_data['fup30_pain_as'] = full_data['fup30_pain_as'].fillna('Null')

In [ ]:
full_data['fup30_pain_as']

In [ ]:
full_data['fup30_pain_as'] = pd.Categorical(full_data.fup30_pain_as)

In [ ]:
full_data['fup30_pain'].unique()

In [ ]:
full_data['fup30_pain_as'].unique()

In [ ]:
fup_pain_as_parsed = []
for data in full_data['fup30_pain_as']:
    if data == 'Null':
        fup_pain_as_parsed.append('Null')
    elif data == 'no pain':
        fup_pain_as_parsed.append('1.0')
    elif data == 'slight pain':
        fup_pain_as_parsed.append('2.0')
    elif data == 'moderate pain':
        fup_pain_as_parsed.append('3.0')
    elif data == 'severe pain':
        fup_pain_as_parsed.append('4.0') 

In [ ]:
full_data['fup30_pain_as_parsed'] = fup_pain_as_parsed

In [ ]:
full_data.drop(['fup30_pain_as'], axis = 1, inplace= True)

In [ ]:
full_data['fup30_pain'] = full_data['fup30_pain'].fillna(full_data['fup30_pain_as_parsed'])

In [ ]:
full_data['fup30_pain'] = pd.Categorical(full_data.fup30_pain)

In [ ]:
full_data['fup30_pain'].unique()

In [ ]:
full_data.isnull().sum()

## Parse ger_grade target variable to ger_grade_flag

In [ ]:
full_data['ger_grade'] = full_data['ger_grade'].fillna('None')

In [ ]:
ger_grade_flag = []
for data in full_data['ger_grade']:
    if data == 'None':
        ger_grade_flag.append('0')
    else:
        ger_grade_flag.append('1')
        

In [ ]:
full_data['ger_grade_flag'] = ger_grade_flag

## Save preprocessed columns as csv

In [ ]:
full_data.to_csv('total_healthcare_parsed.csv' ,sep=',')